In [1]:
!pip install -q torch gtts moviepy opencv-python accelerate git+https://github.com/huggingface/diffusers
!pip -q install diffusers["torch"] transformers
!pip -q install datasets

In [2]:
import os
import torch
# from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, #for automatically loading pre trained model for casual language modelling
    AutoTokenizer, # automatically selecting corresponding valid tokenizer
    # BitsAndBytesConfig, # for quantization configurations
    # HfArgumentParser, #simplifies handling cmd line args for hugging face
    # TrainingArguments, # managing hyperparameters like learning rate, epochs, etc.
    pipeline, # easy-to-use interface for performing various NLP tasks, such as text generation, text classification, question answering, etc.
    # logging, # setting & managing logs. Useful for debug and tracking training.
)
from gtts import gTTS
from moviepy.editor import VideoClip, ImageClip, AudioClip, vfx, concatenate_videoclips, concatenate_audioclips, CompositeAudioClip
import cv2
# from mutagen.wave import WAVE
from diffusers import StableDiffusionPipeline
import shutil

2024-06-30 16:26:14.461488: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-30 16:26:14.461600: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-30 16:26:14.588673: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
system_prompt = "You are a creative writer. Generate engaging short stories based on the provided prompts. Each story should have a clear beginning, middle, and end, and should be imaginative and well-written."
model_1 = "NousResearch/Llama-2-7b-chat-hf"
model_2 = "kulkarni-atharva/Llama-2-7b-storyteller-v2-finetune"
prompt = "A story about hare and the tortoise"

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_2, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_2, trust_remote_code=True)

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

diffusion_pytorch_model.bin:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--redstonehero--cetusmix_v4/snapshots/597680cb66836710a3395ee828bd4968bba5c1ca/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--redstonehero--cetusmix_v4/snapshots/597680cb66836710a3395ee828bd4968bba5c1ca/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--redstonehero--cetusmix_v4/snapshots/597680cb66836710a3395ee828bd4968bba5c1ca/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--redstonehero--cetusmix_v4/snapshots/597680cb66836710a3395ee828bd4968bba5c1ca/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


In [5]:
text_pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=800)
result = text_pipe(f"<s> [INST] <<SYS>> \n {system_prompt} \n <</SYS>> \n \n {prompt} [/INST]")
generated_paragraph_1 = result[0]['generated_text']
print(generated_paragraph_1)
# print(type(generated_paragraph_1))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'

In [ ]:
import gc
del model
del tokenizer
del text_pipe
gc.collect()
gc.collect()

In [15]:
# Sample paragraph as a string
# paragraph = "This is a sample paragraph. [INST]This text should be removed.[/INST] Here is more text. Another [INST]unwanted text here[/INST] to be removed."

# Find the starting and ending indices of the tags
start_tag = "[INST]"
end_tag = "[/INST]"

# Find all occurrences of the tags
start_index = generated_paragraph_1.find(start_tag)
end_index = generated_paragraph_1.find(end_tag)

# Continue removing all instances of the text between the tags until none are found
# while start_index != -1 and end_index != -1:
    # Remove the text between start_tag and end_tag including the tags themselves
paragraph = generated_paragraph_1[:start_index] + generated_paragraph_1[end_index + len(end_tag):]
    
    # Find the next occurrence of the tags
#     start_index = paragraph.find(start_tag)
#     end_index = paragraph.find(end_tag)

# Print the cleaned paragraph

generated_paragraph = paragraph.replace("<newline>", "")
generated_paragraph = generated_paragraph.replace("<s>", "")
print(generated_paragraph)
len(generated_paragraph)
generated_prompts = generated_paragraph.split('.')

  The hare was the fastest animal in the forest. He was always bragging about how fast he was and how he could beat anyone in a race.   The tortoise was tired of hearing the hare's boasts and decided to challenge him to a race. The hare laughed at the tortoise and agreed to race him.   The race began and the hare was off to a flying start. He ran as fast as he could and was so far ahead that he stopped to rest.   The tortoise continued to plod along slowly and steadily. He never stopped to rest and kept on going.   The hare was amazed that the tortoise was catching up to him. He decided to start running again.   The tortoise was still running and was now right behind the hare.   The hare was tired and decided to stop. The tortoise was right behind him.   The hare was amazed that the tortoise was so fast. He decided to challenge him to another race.   The tortoise agreed and the race began. The hare was so fast that he left the tortoise in the dust.   The tortoise continued to plod alon

In [16]:
# import re

# # Define the regex pattern to match text between [INST] and [/INST]
# pattern = r'\[INST\].*?\[\/INST\]'

# # Remove the matched text
# cleaned_paragraph = re.sub(pattern, '', generated_paragraph_1)
# cleaned_paragraph.replace("<s>", "")
# # Print the result
# print(cleaned_paragraph)


2206

In [20]:
from pydub import AudioSegment
audio_clips = []
audio_lengths = []
language="en"
# j = 1
# generated_paragraph = generated_paragraph.split('.')
output_directory = "./audio_files/"

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for i, sentence in enumerate(generated_prompts):
    sentence = sentence.strip()
    speech = gTTS(text=sentence, lang=language, slow=False, tld="com.au")
    filename = os.path.join(output_directory, f"speech_{i}.mp3")
    speech.save(filename)
    audio_clips.append(filename)
    # Load the audio file
    audio_file = AudioSegment.from_file(filename)
    # Get the length of the audio in milliseconds
    audio_length_ms = len(audio_file)
    # Convert milliseconds to seconds
    audio_length_sec = audio_length_ms / 1000
#     audio = WAVE(filename)
#     audio_info = audio.info
#     length = int(audio_info.length)
    audio_lengths.append(audio_length_sec)
#     print(audio_length_sec)

3.432
6.168
6.408
3.624
4.176
5.4
4.176
3.264
4.272
2.76
4.536
3.072
2.64
3.744
3.48
2.952
4.512
4.176
3.264
4.272
2.76
4.536
3.072
2.64
3.744
3.48
2.952
4.512
4.176
3.264
4.272
2.76
4.536
3.072
2.64
3.744
3.48
2.952
4.512
4.176
3.264
3.12


In [ ]:
pipe = StableDiffusionPipeline.from_pretrained("redstonehero/cetusmix_v4", torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.safety_checker = None

In [ ]:
for i in range(1,len(generated_prompts)):
    generated_prompts[i] = generated_prompts[0] + generated_prompts [i]
generated_images = []
# from diffusers import StableDiffusionPipeline
# pipe = StableDiffusionPipeline.from_pretrained("redstonehero/cetusmix_v4", torch_dtype=torch.float16)
# pipe = pipe.to("cuda")
# pipe.safety_checker = None
h=512
w=512
steps=50
guidance=7.5
neg = "easynegative, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worstquality, low quality, normal quality, jpegartifacts, signature, watermark, username, blurry, bad feet, cropped, poorly drawn hands, poorly drawn face, mutation, deformed, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, extra fingers, fewer digits, extra limbs, extra arms,extra legs, malformed limbs, fused fingers, too many fingers, long neck, cross-eyed,mutated hands, polar lowres, bad body, bad proportions, gross proportions, text, error, missing fingers, missing arms, missing legs, extra digit, extra arms, extra leg, extra foot,"
output_directory = "./images/"
os.makedirs(output_directory)
for i, prompt in enumerate(generated_prompts):
  prompt += "8K, ultradetailed"
  image = pipe(prompt, height=h, width=w, num_inference_steps=steps, guidance_scale=guidance, negative_prompt=neg).images[0]
  filename = os.path.join(output_directory , f"image_{i}.jpg")
  image.save(filename)
  generated_images.append(filename)

model_index.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors not found


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

safety_checker/config.json:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--redstonehero--cetusmix_v4/snapshots/597680cb66836710a3395ee828bd4968bba5c1ca/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--redstonehero--cetusmix_v4/snapshots/597680cb66836710a3395ee828bd4968bba5c1ca/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


In [ ]:
video_clips = []
audio_clips_r = []
for i, image in enumerate(generated_images):
    clip = ImageClip(image).set_duration(audio_lengths[i])
    video_clips.append(clip)
for path in audio_clips:
  clip = AudioClip(path)
  audio_clips_r.append(clip)
final_video = concatenate_videoclips(video_clips)
# final_audio = concatenate_audioclips(audio_clips)
final_video.audio = CompositeAudioClip(audio_clips)
os.makedirs("./video/")
final_video.write_videofile("./video/final_video.mp4")

In [5]:
def delete_folder(folder_path):
    try:
        shutil.rmtree(folder_path)
        print(f"Deleted folder: {folder_path}")
    except Exception as e:
        print(f"Failed to delete folder: {folder_path}. Error: {e}")


# Danger Zone:
Running the following cells will delete all the outputs including the generated video:

In [ ]:
folder_path = ["/kaggle/working/audio_files", "/kaggle/working/images", "/kaggle/working/video"]
for path in folder_path:
    delete_folder(path)

In [6]:
delete_folder("/kaggle/working/audio_files")

Deleted folder: /kaggle/working/audio_files
